In [1]:
import tensorflow as tf
from keras.datasets import cifar10
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization import *

### Loading CIFAR-10 dataset

In [2]:
(trainX, trainy), (testX, testy) = cifar10.load_data()

In [3]:
# print to make sure we have the correct shapes + number of images for training
print("number of train pictures:", trainX.shape)
print("number of trained picture values:", trainy.shape)
# divide by 255 to make [0,255] into [0,1] + print to make sure!
trainy = tf.keras.utils.to_categorical(trainy,10)
testy = tf.keras.utils.to_categorical(testy,10)
trainX = trainX/255.0
testX = testX/255.0

number of train pictures: (50000, 32, 32, 3)
number of trained picture values: (50000, 1)


### Defining the quantization config
`DefaultDenseQuantizeConfig` is 8 bit

`ModifiedDenseQuantizeConfig` is 4 bit

`UltraDenseQuantizeConfig` is 2 bit

### VGG-16 model

In [1]:
# example of loading the vgg16 model
from keras.applications.vgg16 import VGG16
# load model
model = VGG16()
# summarize the model
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### VGG-16 Clone Without Quantization

In [4]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
import numpy as np
vgg_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights=None)
vgg_model.trainable = True
model_wq = tf.keras.Sequential()
#block-1
model_wq.add(Conv2D(input_shape=(32,32,3),
                    filters=64,kernel_size=(3,3),
                    padding="same", 
                    activation="relu",
                    name='block1_conv1'))
model_wq.add(Conv2D(filters=64,
                    kernel_size=(3,3),
                    padding="same", 
                    activation="relu",
                    name='block1_conv2'))
model_wq.add(MaxPool2D(pool_size=(2,2), strides=(2,2), name='block1_pool'))


#block-2
model_wq.add(Conv2D(filters=128, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block2_conv1'))
model_wq.add(Conv2D(filters=128, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block2_conv2'))
model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block2_pool'))

#block-3
model_wq.add(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv1'))
model_wq.add(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv2'))
model_wq.add(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv3'))
model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block3_pool'))

#block-4
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv1'))
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv2'))
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv3'))
model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block4_pool'))

#block-5
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv1'))
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv2'))
model_wq.add(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv3'))
model_wq.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block5_pool'))

#fc1, fc2 and predictions
model_wq.add(Flatten(name='flatten'))
model_wq.add(Dense(units=4096,activation="relu",name='fc1'))
model_wq.add(Dense(units=4096,activation="relu",name='fc2'))
model_wq.add(Dense(units=1000, activation="softmax",name='predictions'))

model_wq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 8, 8, 256)         2

In [5]:
# Compile the model
model_wq.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model_wq.fit(trainX[:1000], trainy[:1000],
          batch_size=50,
          epochs=15,
          validation_split=0)

Epoch 1/15


ValueError: in user code:

    /home/darkboy/anaconda3/envs/smartsight/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/darkboy/anaconda3/envs/smartsight/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/darkboy/anaconda3/envs/smartsight/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/darkboy/anaconda3/envs/smartsight/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/darkboy/anaconda3/envs/smartsight/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(

In [4]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

class ModifiedDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=4, symmetric=True, narrow_range=False, per_axis=False))]

    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=4, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

class UltraDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=2, symmetric=True, narrow_range=False, per_axis=False))]

    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=2, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}
